# Deep Learning - Project

## Parking Space Detection

### Group 02

# 0. Imports & Global Constants

In [ ]:
import os
import random
import shutil
import xml.etree.ElementTree as ET

ORIGINAL_DATASET_PATH = "./datasets/original"
TRANSFORMED_DATASET_PATH = "./datasets/transformed"
TRAINING_DATASET_PATH = "./datasets/training"
VALIDATION_DATASET_PATH = "./datasets/validation"
TESTING_DATASET_PATH = "./datasets/testing"

TRAINING_RATIO = 0.8
VALIDATION_RATIO = 0.15
TESTING_RATIO = 0.05

GITKEEP = ".gitkeep"

IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 720

CLASS_MAPPING = {"0": 0, "1": 1}

# 1. Data Pipeline

# 1.1. Transforming

First we start by transforming our original dataset

In [ ]:
for root, _, files in os.walk(ORIGINAL_DATASET_PATH):
    for file in files:
        if file == GITKEEP:
            continue
        source = os.path.join(root, file)
        destination = os.path.join(TRANSFORMED_DATASET_PATH, file)
        shutil.move(source, destination)

for dir in os.listdir(ORIGINAL_DATASET_PATH):
    if not os.path.isdir(dir):
        continue
    path = os.path.join(ORIGINAL_DATASET_PATH, dir)
    shutil.rmtree(path)

print("Merge complete!")

## 1.2. Create the Labels

We'll convert the labels from XML to TXT

In [ ]:
xml_files = [
    file for file in os.listdir(TRANSFORMED_DATASET_PATH) if file.endswith(".xml")
]

for xml_file in xml_files:
    xml_path = os.path.join(TRANSFORMED_DATASET_PATH, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    txt_filename = os.path.splitext(xml_file)[0] + ".txt"
    txt_path = os.path.join(TRANSFORMED_DATASET_PATH, txt_filename)
    with open(txt_path, "w") as txt_file:
        for space in root.findall("space"):
            occupied = space.get("occupied")
            class_index = CLASS_MAPPING.get(occupied, -1)
            if class_index == -1:
                continue
            rotated_rect = space.find("rotatedRect")
            center = rotated_rect.find("center")
            size = rotated_rect.find("size")
            x = float(center.get("x")) / IMAGE_WIDTH
            y = float(center.get("y")) / IMAGE_HEIGHT
            w = float(size.get("w")) / IMAGE_WIDTH
            h = float(size.get("h")) / IMAGE_HEIGHT
            txt_file.write(f"{class_index} {x} {y} {w} {h}\n")
    os.remove(xml_path)

print("Conversion complete!")

# Train Test Split

In [ ]:
image_files = [f for f in os.listdir(TRANSFORMED_DATASET_PATH) if f.endswith(".jpg")]
print(image_files)

num_samples = len(image_files)
num_train = int(TRAINING_RATIO * num_samples)
num_test = int(TESTING_RATIO * num_samples)
num_val = num_samples - num_train - num_test

random.shuffle(image_files)

train_files = image_files[:num_train]
test_files = image_files[num_train:num_train + num_test]
val_files = image_files[num_train + num_test:]

# Move corresponding txt files along with images
for folder, files in [(TRAINING_DATASET_PATH, train_files), (TESTING_DATASET_PATH, test_files), (VALIDATION_DATASET_PATH, val_files)]:
    for file in files:
        # Move image file
        source_image_path = os.path.join(TRANSFORMED_DATASET_PATH, file)
        destination_image_path = os.path.join(folder + '/images', file)
        shutil.move(source_image_path, destination_image_path)
        
        # Move corresponding txt file
        txt_file = os.path.splitext(file)[0] + ".txt"
        source_txt_path = os.path.join(TRANSFORMED_DATASET_PATH, txt_file)
        destination_txt_path = os.path.join(folder + '/labels', txt_file)
        shutil.move(source_txt_path, destination_txt_path)

print("Data split into train, test, and val sets.")